# **Deploying The Dataset directly from kaggle**

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/Kaggle_dataset'

In [ ]:
!kaggle datasets download -d akash2907/bird-species-classification

100% 1.37G/1.37G [00:52<00:00, 22.4MB/s]
100% 1.37G/1.37G [00:52<00:00, 28.0MB/s]


In [ ]:
!unzip '/content/drive/MyDrive/Kaggle_dataset/bird-species-classification.zip'

Archive:  /content/drive/MyDrive/Kaggle_dataset/bird-species-classification.zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/blasti/DSC_6411.jpg  
  inflating: test_data/test_data/bonegl/DSC_4587.jpg  
  inflating: test_data/test_data/bonegl/

# **Building VGG16 model**

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np

In [ ]:
train = '/content/train_data'
test = '/content/test_data'

In [ ]:
train_gen = ImageDataGenerator(rescale=(1./255), shear_range=0.2, horizontal_flip=True, vertical_flip=True)
test_gen = ImageDataGenerator(rescale=(1./255))

In [ ]:
train = train_gen.flow_from_directory(train, batch_size=5, class_mode='categorical', target_size=(224, 224))
test = test_gen.flow_from_directory(test, batch_size=5, class_mode='categorical', target_size=(224, 224))

Found 150 images belonging to 1 classes.
Found 157 images belonging to 1 classes.


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [ ]:
# Add preprocessed layers to the vgg
vgg = VGG16(include_top=False, input_shape=(224, 224, 3), weights='imagenet')

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
# Train the model with existing weights
for layers in vgg.layers:
  layers.trainable=False

In [ ]:
x = Flatten()(vgg.output) # Flatten the output from the VGG16

In [ ]:
# Output layer
prediction = Dense(4, activation='Softmax')(x)

In [ ]:
# Create VGG16 model
model= Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
model.fit(train, epochs=50, steps_per_epoch=len(train), validation_steps=len(test))

Epoch 1/50
30/30 [==============================] - 122s 4s/step - loss: 6.2440 - accuracy: 0.0333
Epoch 2/50
30/30 [==============================] - 120s 4s/step - loss: 6.2745 - accuracy: 0.0600
Epoch 3/50
30/30 [==============================] - 123s 4s/step - loss: 6.2354 - accuracy: 0.1333
Epoch 4/50
30/30 [==============================] - 117s 4s/step - loss: 6.3033 - accuracy: 0.0733
Epoch 5/50
30/30 [==============================] - 119s 4s/step - loss: 6.3217 - accuracy: 0.1067
Epoch 6/50
30/30 [==============================] - 121s 4s/step - loss: 6.2644 - accuracy: 0.1067
Epoch 7/50
30/30 [==============================] - 120s 4s/step - loss: 6.2801 - accuracy: 0.1267
Epoch 8/50
30/30 [==============================] - 118s 4s/step - loss: 6.3345 - accuracy: 0.0800
Epoch 9/50
30/30 [==============================] - 120s 4s/step - loss: 6.3936 - accuracy: 0.1867
Epoch 10/50
30/30 [==============================] - 115s 4s/step - loss: 6.3988 - accuracy: 0.1200
Epoch 11/

# **Building ResNet50 model**


In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

In [ ]:
for layer in resnet.layers:
  layer.trainable=False

In [ ]:
x=Flatten()(resnet.output)

In [ ]:
out = Dense(4, activation='softmax')(x)

In [ ]:
resnet_model = Model(inputs=resnet.input,outputs=out)

In [ ]:
resnet_model.summary()

In [ ]:
resnet_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
resnet_model.fit(train,epochs=50,validation_data=test,steps_per_epoch=len(train),validation_steps=len(test))

Epoch 1/50
30/30 [==============================] - 208s 7s/step - loss: 6.1967 - accuracy: 0.9867 - val_loss: 6.2265 - val_accuracy: 0.9108
Epoch 2/50
30/30 [==============================] - 151s 5s/step - loss: 6.2048 - accuracy: 0.9267 - val_loss: 6.0586 - val_accuracy: 0.3885
Epoch 3/50
30/30 [==============================] - 147s 5s/step - loss: 6.2096 - accuracy: 0.8867 - val_loss: 6.3614 - val_accuracy: 0.8917
Epoch 4/50
30/30 [==============================] - 144s 5s/step - loss: 6.2475 - accuracy: 0.7733 - val_loss: 6.2539 - val_accuracy: 0.9873
Epoch 5/50
30/30 [==============================] - 146s 5s/step - loss: 6.2355 - accuracy: 0.7933 - val_loss: 6.2515 - val_accuracy: 0.9108
Epoch 6/50
30/30 [==============================] - 208s 7s/step - loss: 6.2317 - accuracy: 0.8800 - val_loss: 6.1777 - val_accuracy: 0.8471
Epoch 7/50
30/30 [==============================] - 145s 5s/step - loss: 6.3063 - accuracy: 0.8000 - val_loss: 6.1114 - val_accuracy: 0.0828
Epoch 8/50
30